In [0]:
%sql
-- VIEW OF TABLES
select * from workspace.default.flagged_rides limit 5;

week,zip,fare_amount,trip_distance
2016-01-25T00:00:00.000Z,11109,52.0,3.0
2016-02-15T00:00:00.000Z,7311,60.0,2.0
2016-01-25T00:00:00.000Z,10065,52.0,1.12
2016-02-08T00:00:00.000Z,11422,52.0,0.2
2016-01-11T00:00:00.000Z,11422,52.0,8.7


In [0]:
%sql
-- VIEW OF TABLES
select * from workspace.default.taxi_raw_records limit 5;

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-13T21:47:53.000Z,2016-02-13T21:57:15.000Z,1.4,8.0,10103,10110
2016-02-13T18:29:09.000Z,2016-02-13T18:37:23.000Z,1.31,7.5,10023,10023
2016-02-06T19:40:58.000Z,2016-02-06T19:52:32.000Z,1.8,9.5,10001,10018
2016-02-12T19:06:43.000Z,2016-02-12T19:20:54.000Z,2.3,11.5,10044,10111
2016-02-23T10:27:56.000Z,2016-02-23T10:58:33.000Z,2.6,18.5,10199,10022


In [0]:
%sql
-- VIEW OF TABLES
select * from workspace.default.top_n limit 5;

week,avg_amount,avg_distance,fare_amount,trip_distance,zip
2016-01-04T00:00:00.000Z,11.91,2.865,95.0,5.2,10009
2016-02-15T00:00:00.000Z,12.24,2.894,60.0,2.0,7311
2016-02-22T00:00:00.000Z,12.79,2.973,60.0,0.92,11422


In [0]:
%sql
-- VIEW OF TABLES
select * from workspace.default.weekly_stats limit 5;

week,avg_amount,avg_distance
2016-02-01T00:00:00.000Z,11.990339116719243,2.7463367507886396
2016-01-18T00:00:00.000Z,11.966793403573064,2.7421759047182723
2016-02-15T00:00:00.000Z,12.244146522870956,2.8944923763480825
2016-01-04T00:00:00.000Z,11.907765076862436,2.864603862830116
2016-02-08T00:00:00.000Z,12.20651356238698,2.751081374321874


In [0]:
%sql
-- VALIDATION 1: Bronze table should NOT contain trip_distance <= 0
-- This proves the DLT expectation worked correctly.
SELECT COUNT(*) AS invalid_distance_rows
FROM workspace.default.taxi_raw_records
WHERE trip_distance <= 0;


invalid_distance_rows
0


In [0]:
%sql
-- VALIDATION: All flagged rides must satisfy at least one suspicious-ride rule.
-- If ANY rows appear in the output, your flagged_rides table contains incorrect entries.

SELECT fr.*
FROM workspace.default.flagged_rides fr
LEFT JOIN workspace.default.taxi_raw_records raw
  ON fr.week = date_trunc("week", raw.tpep_pickup_datetime)
 AND fr.zip = raw.pickup_zip
 AND fr.fare_amount = raw.fare_amount
 AND fr.trip_distance = raw.trip_distance
WHERE NOT (
      (raw.fare_amount > 50 AND raw.pickup_zip = raw.dropoff_zip)
   OR (raw.fare_amount > 50 AND raw.trip_distance < 5)
);


week,zip,fare_amount,trip_distance


In [0]:
%sql
-- VALIDATION 3: Compare weekly average fares to raw data to ensure correct aggregation.
-- If both averages match for each week, weekly_stats is correct.
SELECT 
  ws.week,
  ws.avg_amount AS silver_avg_amount,
  ROUND(AVG(raw.fare_amount), 2) AS recomputed_avg_amount
FROM workspace.default.weekly_stats ws
JOIN workspace.default.taxi_raw_records raw
  ON ws.week = date_trunc("week", raw.tpep_pickup_datetime)
GROUP BY ws.week, ws.avg_amount
ORDER BY ws.week;


week,silver_avg_amount,recomputed_avg_amount
2015-12-28T00:00:00.000Z,12.178038379530918,12.18
2016-01-04T00:00:00.000Z,11.907765076862436,11.91
2016-01-11T00:00:00.000Z,12.332039911308204,12.33
2016-01-18T00:00:00.000Z,11.966793403573064,11.97
2016-01-25T00:00:00.000Z,12.981361426256077,12.98
2016-02-01T00:00:00.000Z,11.990339116719243,11.99
2016-02-08T00:00:00.000Z,12.20651356238698,12.21
2016-02-15T00:00:00.000Z,12.244146522870956,12.24
2016-02-22T00:00:00.000Z,12.79211403184006,12.79
2016-02-29T00:00:00.000Z,12.60960960960961,12.61


In [0]:
%sql
-- VALIDATION 4: Gold table should show the top 3 highest-fare flagged rides.
-- If both result sets match, then the Gold logic is correct.
SELECT *
FROM workspace.default.flagged_rides
ORDER BY fare_amount DESC
LIMIT 3;


week,zip,fare_amount,trip_distance
2016-01-04T00:00:00.000Z,10009,95.0,5.2
2016-02-15T00:00:00.000Z,7311,60.0,2.0
2016-02-22T00:00:00.000Z,11422,60.0,0.92
